In [ ]:
from google.colab import drive 
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import os
os.chdir("drive/MyDrive/Cognizant_final_hackathon")

## Installing and importing

In [ ]:
## Go down and install before starting

In [ ]:
# install textract

from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
import pandas as pd 
import re
import textract
stop_words = set(stopwords.words('english')) 



# text = textract.process('topic_7/1002.2194v1.Genetic_Algorithm_Based_Optimization_of_Clustering_in_Ad_Hoc_Networks.pdf', method='pdfminer')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Text cleaning

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [ ]:
def text_cleaning(text):
  newString = text.lower()
  #newString = text
  newString = BeautifulSoup(newString, "lxml").text

  #extracting the starting 5000 letters from abstract and introduction
  try:
    index_abstract = newString.index('abstract')  # abstraction index
  except:
    index_abstract = -1

  try:
    index_introduction = newString.index('introduction')  # introduction index
  except:
    index_introduction = -1


  if index_abstract < index_introduction:                      
    starting_txt = newString[index_abstract+9:index_abstract+5009]  # adding 9 so that w0rd abstract is selected

  elif index_abstract > index_introduction:
    starting_txt = newString[index_introduction+12 : index_introduction+5012] # adding 12 so that w0rd abstract is not selected
  
  else:
    starting_txt = newString[:5000]            # i.e. neither abstract for introduction is present




  #extracting conclusion
  try:
    index_conclusion = newString.rindex('conclusion')  #last index of conclusion
  except:
    index_conclusion = -1
 
  # Selection the part after the Conclusion heading (onlu conclusion)(selects the conclusion if its contained only within one paragraph)
  conclusion_txt=''
  if (index_conclusion > 0):
    temp = newString[index_conclusion:]
    i=0
    try:
      while (len(conclusion_txt) <80):
        conclusion_txt = temp.split("\n\n")[i]
        i+=1 
    except:
      pass


# trying to maximize the words in the cases where the conclusion is in many paragraphs.
  if (len(conclusion_txt) < 300):
    conclusion_txt = newString[index_conclusion+11: index_conclusion+511 ]

  newString = starting_txt + conclusion_txt
  newString = re.sub(r'\([^)]*\)', '', newString)                  # remove any text inside brackets
  newString = re.sub('[0-9]','', newString)                        # remove the digits
  newString = re.sub(r"[\([{})\]]", "", newString)                 # remove any kind of extra brackets  
  newString = re.sub('"','', newString)
  newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])
  newString = re.sub(r"'s\b","",newString)
  #newString = re.sub("[^a-zA-Z]", " ", newString)                 # removes , . ; ...         
  newString = re.sub('[m]{2,}', 'mm', newString)

  #tokens = [w for w in newString.split() if not w in stop_words]  # removing stopwords

  tokens = [w for w in newString.split()]                          # not removing stopwords
  long_words=[]
  for i in tokens:
    if len(i)>1:                                                 #removing short words
      long_words.append(i)   
  str_1 = ''
  for word in long_words:
    str_1 = str_1+word+' '
  return(str_1)

## getting pdf file names

In [ ]:
def get_files(topic):
  paths = []
  local_download_path = str(topic)
  for filename in os.listdir(local_download_path):
    if ((filename.endswith("pdf") or filename.endswith("txt")) and filename not in paths): 
      paths.append(filename)

  return paths

## Algorithms

### LSA

In [ ]:
# install sumy
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.tokenizers import Tokenizer as sum_tokenizer
from sumy.parsers.plaintext import PlaintextParser as sum_parser
import nltk
nltk.download('punkt')

def fun_lsa(text):
  parser=sum_parser.from_string(text,sum_tokenizer('english'))

  # creating the summarizer
  lsa_summarizer=LsaSummarizer()
  lsa_summary= lsa_summarizer(parser.document,4)
  temp=''
  # Printing the summary
  for sentence in lsa_summary:
      temp = temp + str(sentence) + ' '
  return temp

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Lex

In [ ]:
# from sumy.summarizers.lex_rank import LexRankSummarizer
# def fun_lex(text):
#   parser=sum_parser.from_string(text,sum_tokenizer('english'))
#   lex_rank_summarizer = LexRankSummarizer()
#   lexrank_summary = lex_rank_summarizer(my_parser.document,sentences_count=6)

#   # Printing the summary
#   for sentence in lexrank_summary:
#     summary = summary + str(sentence) + ' ' 
#   return summary

### KL

In [ ]:
from sumy.summarizers.kl import KLSummarizer
def fun_kl(text):
  parser = sum_parser.from_string(text,sum_tokenizer('english'))
  kl_summarizer = KLSummarizer()
  kl_summary = kl_summarizer(parser.document,sentences_count=50)
  summary = ''
  for sentence in kl_summary:
    summary = summary + str(sentence) + ' ' 
  return summary

### T5

In [ ]:
# install transformer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_t5 = AutoModelForSeq2SeqLM.from_pretrained("t5-large")
tokenizer_t5 = AutoTokenizer.from_pretrained("t5-large")

# T5 uses a max_length of 512 so we have to cut it

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [ ]:
def fun_t5(text):
  inputs = tokenizer_t5("summarize: " +text , return_tensors="pt", max_length=512, truncation=True)
  outputs = model_t5.generate(inputs["input_ids"], max_length=150, min_length=70, length_penalty=3.0, num_beams=4, early_stopping=True)
  return (tokenizer_t5.decode(outputs[0]))


### Bart

In [ ]:
# # install torch
# from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig
# import torch
# tokenizer_bart = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
# model_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
# torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
# def fun_bart(text):
#   article_input_ids = tokenizer_bart.batch_encode_plus([text], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
#   summary_ids = model_bart.generate(article_input_ids,
#                               num_beams=4,
#                               length_penalty=3.0,
#                               max_length=300,
#                               no_repeat_ngram_size=3)

#   summary_txt = tokenizer_bart.decode(summary_ids.squeeze(), skip_special_tokens=True)
#   return (summary_txt)

### Bert

In [ ]:
from summarizer import Summarizer
bert_model = Summarizer()
#ext_summary = bert_model(extractive_summary, num_sentences=10)

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

### Gpt-2

In [ ]:
# # Importing model and tokenizer
# from transformers import GPT2Tokenizer,GPT2LMHeadModel

# # Instantiating the model and tokenizer with gpt-2
# tokenizer_gpt2 = GPT2Tokenizer.from_pretrained('gpt2')
# model_gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
# def fun_gpt_2(text):
#   # Encoding text to get input ids & pass them to model.generate()
#   inputs=tokenizer_gpt2.batch_encode_plus([text],return_tensors='pt',max_length=512)
#   summary_ids=model_gpt2.generate(inputs['input_ids'],max_length=300,early_stopping=True)
#   GPT_summary=tokenizer_gpt2.decode(summary_ids[0],skip_special_tokens=True)
#   return GPT_summary

## Part_1

In [ ]:
%%time
def fun_part_1(files,topic,iteration):                       # args: (list of file names, topic number, summary number)
  i = 0
  ext_sum=''
  extractive_summary = ''
  abstractive_summary=''
  print("Inside part_1:{}".format(topic))
  text=''
  counter = 0
  txt_files=[]

  if (topic == "topic_6"):
    print("inside")
    df = files.copy()
    for index, row in df.iterrows():
      i+=1
      text = text + row[0]
      if (len(text) >= 5000):
        counter+=1
        clean_text = text_cleaning(text)
        extracted_text =''
        extracted_text = fun_lsa(clean_text)                               #lsa
        extractive_summary = extractive_summary + extracted_text

      if (counter == 30):
        counter = 0
        ext_sum = ext_sum + extractive_summary + ' '                             # ext_sum, to be saved in disk (full extractive summary)
        abstractive_text=''
        try:
          #abstractive_text = fun_gpt_2(extractive_summary)                      #gpt 
          abstractive_text = bert_model(extractive_summary, num_sentences=10)    #bert
          #abstractive_text = fun_bart(extractive_summary)                       #bart 
        except:
          print("error in numbers: {} to {}".format(i, (i+200)))
        text=''
        extractive_summary = ''                                                   # resetting the extractive summary again for next loop
        abstractive_summary = abstractive_summary + abstractive_text + ' '
      elif (df.shape[0] == i):
        ext_sum = ext_sum + extractive_summary + ' '
        abstractive_text=''
        try:
          #abstractive_text = fun_gpt_2(extractive_summary)                      #gpt 
          abstractive_text = bert_model(extractive_summary, num_sentences=10)    #bert
          #abstractive_text = fun_bart(extractive_summary)                       #bart 
        except:
          print("error in numbers: {} to {}".format(i, (i+10)))
        extractive_summary = ''
        abstractive_summary = abstractive_summary + abstractive_text + ' '
      if (i%1000 == 0):
        print(i)
  else:
    for file_name in files:
      if file_name[-3:] == "txt":
        txt_files.append(file_name)
    
    pdf_files = list(set(files)-set(txt_files))

    print("No of pdfs:{}".format(len(pdf_files)))
    print("No of txts:{}".format(len(txt_files)))

  ##
  ## Reading texts
  ##

    for file_name in txt_files:
      try:
        i+=1
        path = str(topic + '/' + file_name)
        file1 = open(path,"r")
        tmp = file1.read()
        text = text + tmp
        if (i%40 == 0):
          clean_text = text_cleaning(text)
          extracted_text =''
          extracted_text = fun_lsa(clean_text)                               #lsa
          extractive_summary = extractive_summary + extracted_text
          if (len(extractive_summary) == 0):
            print("File No:{}, file:{}".format(i,file_name))
          text =''                                                              # resetting text for next loop
      except: 
        print('error in file:{}, path:{}'.format(i,file_name))
      
      if (i%200 == 0):
        ext_sum = ext_sum + extractive_summary + ' '
        abstractive_text=''
        try:
          #abstractive_text = fun_gpt_2(extractive_summary)                      #gpt 
          abstractive_text = bert_model(extractive_summary, num_sentences=10)    #bert
          #abstractive_text = fun_bart(extractive_summary)                       #bart 
        except:
          print("error in numbers: {} to {}".format(i, (i+200)))
        extractive_summary = ''                                                   # resetting the extractive summary again for next loop
        abstractive_summary = abstractive_summary + abstractive_text + ' '
      
      elif (len(files) - i == 0):
        ext_sum = ext_sum + extractive_summary + ' '
        abstractive_text=''
        try:
          #abstractive_text = fun_gpt_2(extractive_summary)                      #gpt 
          abstractive_text = bert_model(extractive_summary, num_sentences=10)    #bert
          #abstractive_text = fun_bart(extractive_summary)                       #bart 
        except:
          print("error in numbers: {} to {}".format(i, (i+10)))
        extractive_summary = ''
        abstractive_summary = abstractive_summary + abstractive_text + ' '

  ##
  ## Reading pdfs
  ##
    for file_name in pdf_files:
      try:
        i+=1
        path = str(topic + '/' + file_name)
        text = textract.process(path, method='pdfminer')
        clean_text = text_cleaning(text)
        extracted_text =''
        extracted_text = fun_lsa(clean_text)                               #lsa
        extractive_summary = extractive_summary + extracted_text
        if (len(extractive_summary) == 0):
          print("File No:{}, file:{}".format(i,file_name))
          
      except: 
        print('error in file:{}, path:{}'.format(i,file_name))
      
      if (i%20 == 0):
        ext_sum = ext_sum + extractive_summary + ' '
        abstractive_text=''
        try:
          #abstractive_text = fun_gpt_2(extractive_summary)                      #gpt 
          abstractive_text = bert_model(extractive_summary, num_sentences=10)    #bert
          #abstractive_text = fun_bart(extractive_summary)                       #bart 
        except:
          print("error in numbers: {} to {}".format(i, (i+200)))
        extractive_summary = ''                                                   # resetting the extractive summary again for next loop
        abstractive_summary = abstractive_summary + abstractive_text + ' '
      
      elif (len(files) - i == 0):
        ext_sum = ext_sum + extractive_summary + ' '
        abstractive_text=''
        try:
          #abstractive_text = fun_gpt_2(extractive_summary)                      #gpt 
          abstractive_text = bert_model(extractive_summary, num_sentences=10)    #bert
          #abstractive_text = fun_bart(extractive_summary)                       #bart 
        except:
          print("error in numbers: {} to {}".format(i, (i+10)))
        extractive_summary = ''
        abstractive_summary = abstractive_summary + abstractive_text + ' '

  op_file_1 = "outputs/"+str(topic)+"_abs_summary"+ str(iteration)+ ".txt"
  op_file_2 = "outputs/"+str(topic)+"_ext_summary"+ str(iteration)+ ".txt"

  file1 = open(op_file_1,"w")
  file1.write(abstractive_summary)
  file1.close()

  file2 = open(op_file_2,"w")
  file2.write(ext_sum)
  file2.close()
  return abstractive_summary

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs


## Part_2

In [ ]:
%%time
# implementing final
def fun_part_2(abs_summary,topic):

  kl_abs = fun_kl(abs_summary)  

  words = kl_abs.split()
  i=0
  text = ''
  final_abstract=''
  n = int(len(words))

  if (n > 600):
    for word in words:
      if (i<501):                                       
        i+=1
        text = text + word + " "
      else:
        print('inside else')
        abstract = fun_t5(text)
        text = ''
        i=0
        final_abstract = final_abstract + abstract + ' '                        #t5
        print('#'* 100)
        print(abstract)
        
    if (len(text) > 100):           # for remaining words that are left after 500 and are move than 100 words
      print('Inside if')
      abstract = fun_t5(text)
      final_abstract = final_abstract + abstract + ' '
      print('#'* 100)
      print(abstract)

  else:                             # if summary from kl is less than 500 words
    abstract = fun_t5(text)
    final_abstract = final_abstract + abstract + ' '
  
  
  op_file_1 = "outputs/"+str(topic)+"_8_final_summary.txt"

  file1 = open(op_file_1,"w")
  file1.write(final_abstract)
  file1.close()

  return kl_abs, final_abstract


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


# topic_1

In [ ]:
dict_topic = {"topic_1":"pdf", 
              "topic_2":"pdf",
              "topic_3":"txt",
              "topic_4":"txt",
              "topic_5":"txt",
              "topic_6":"csv",
              "topic_7":"pdf",
              "topic_8":"pdf",
              "topic_9":"pdf",
              "topic_10":"pdf",
              "topic_11":"pdf",
              "topic_12":"pdf",
              "topic_13":"pdf",
              "topic_14":"pdf",
              "topic_15":"pdf",
              }

In [ ]:
%%time
for topic, file_type in dict_topic.items():
  if (topic == "topic_1"):                                   ##  change it one topic at a time
    print("doing:{}".format(topic))
    files = get_files(topic)
    print("len files:{}".format(len(files)))
    abs_summary = fun_part_1(files, topic, 6)
    print("Abs_summary done")   
    kl_abs , final_abs = fun_part_2(abs_summary,topic)
    print("Completed")

  elif (topic == "topic_6"):
    print("doing:{}".format(topic))
    files = pd.read_csv('topic_6/topic_6.csv')
    abs_summary = fun_part_1(files, topic, 8)
    print("Abs_summary done")   
    kl_abs , final_abs = fun_part_2(abs_summary,topic)
    print("Completed")



## PIP

In [ ]:
% pip install sentencepiece
% pip install sumy
% pip install textract
% pip install bert-extractive-summarizer
% pip install torch
% pip install transformers

     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 83 kB 1.8 MB/s 
     |████████████████████████████████| 10.1 MB 25.8 MB/s 
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21711 sha256=fb823519bd7018cc1ef8af55569ef86dce2e002039a13adca2c44f1397f4b23e
  Stored in directory: /root/.cache/pip/wheels/d4/bf/51/81d27ad638e1a6dca4f362ecc33d1e2c764b8ea7ec751b8fc1
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746883 sha256=3b535b4d2653ab159e29f919505d2de8f07efff0fa49b65307028a25f3456aaa
  Stored in directory: /root/.cache/pip/wheels/57/e8/3f/120ccc1ff7541c108bc5d656e2a14c39da0d824653b62284c6
Successfully built breadability pycountry
     |████████████████████████████████| 106 kB 9.3 MB/s 
     |████████████████████████████████| 111 kB 42.7 MB/s 
     |████████████████████████████████| 69 kB 6.2 MB/s 
     |████████████████████████████████| 5.6 MB 22.8 MB/s 
     |██████

     |████████████████████████████████| 2.9 MB 6.8 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 636 kB 32.9 MB/s 
     |████████████████████████████████| 895 kB 37.5 MB/s 
     |████████████████████████████████| 3.3 MB 33.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
function KeepClicking(){
console.log("Clicking");
document.querySelector("colab-connect-button").click()
}
setInterval(KeepClicking,60000)